### Логистическая регрессия и полезные преобразования

Данные о доходах населения.  
Источник данных: https://www.kaggle.com/uciml/adult-census-income

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('./data/adult.csv') # Загружаем данные
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [3]:
# Заменим ? на NaN
df = df.replace('?',np.nan)

In [4]:
# Узнаем где есть пропуски
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       30725 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      30718 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  31978 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [5]:
# Удалим строки с пропусками
df = df.dropna()

In [6]:
df['income'].value_counts()

<=50K    22654
>50K      7508
Name: income, dtype: int64

#### Признаки

Когда мы говорим о категориальных данных, мы должны различать **номинальные** и **порядковые признаки**.              
***Порядковые признаки*** могут пониматься как категориальные значения, которые могут быть отсортированы или упорядочены.   
***Hоминальные признаки*** не подразумевают порядка.

Нужно помнить, что метки классов не являются порядковыми, и поэтому не имеет значения, какое целое число мы присваиваем отдельно взятой строковой метке.

In [7]:
# Какой это признак?
df['education'].value_counts()

HS-grad         9840
Some-college    6678
Bachelors       5044
Masters         1627
Assoc-voc       1307
11th            1048
Assoc-acdm      1008
10th             820
7th-8th          557
Prof-school      542
9th              455
12th             377
Doctorate        375
5th-6th          288
1st-4th          151
Preschool         45
Name: education, dtype: int64

In [8]:
df['education.num'].value_counts()

9     9840
10    6678
13    5044
14    1627
11    1307
7     1048
12    1008
6      820
4      557
15     542
5      455
8      377
16     375
3      288
2      151
1       45
Name: education.num, dtype: int64

In [9]:
del df['education']

In [10]:
y = df['income'].replace({'<=50K': 0, '>50K': 1})   # Ответы 

In [11]:
# Перекодируем в бинарные признаки
df['sex'] = df['sex'].replace({'Female': 0, 'Male': 1})
df['race'] = df['race'].replace({'White': 0, 'Black': 1})

X = df.drop('income', axis=1)                      # Признаки

In [12]:
X

,age,workclass,fnlwgt,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
1,82,Private,132870,9,Widowed,Exec-managerial,Not-in-family,0,0,0,4356,18,United-States
3,54,Private,140359,4,Divorced,Machine-op-inspct,Unmarried,0,0,0,3900,40,United-States
4,41,Private,264663,10,Separated,Prof-specialty,Own-child,0,0,0,3900,40,United-States
5,34,Private,216864,9,Divorced,Other-service,Unmarried,0,0,0,3770,45,United-States
6,38,Private,150601,6,Separated,Adm-clerical,Unmarried,0,1,0,3770,40,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Private,310152,10,Never-married,Protective-serv,Not-in-family,0,1,0,0,40,United-States
32557,27,Private,257302,12,Married-civ-spouse,Tech-support,Wife,0,0,0,0,38,United-States
32558,40,Private,154374,9,Married-civ-spouse,Machine-op-inspct,Husband,0,1,0,0,40,United-States
32559,58,Private,151910,9,Widowed,Adm-clerical,Unmarried,0,0,0,0,40,United-States


#### Рассмотрим пример
Пусть у нас есть данные, в которых есть **признак** "Цвет". В столбце 3 значения: синий, зеленый, красный. Кодируем:

синий —> 0  
зеленый —> 1   
красный —> 2  


Сейчас мы сделали одну из наиболее распространенных ошибок, которая встречается во время 
работы с категориальными данными. Можете определить, в чем проблема?

Несмотря на то, что значения цвета не идут в том или ином конкретном порядке, алгоритм обучения предположит, что зеленый больше синего, а красный больше зеленого. Хотя это допущение является неправильным, алгоритм все равно способен произвести полезные результаты. Однако эти результаты не будут оптимальными.

#### Прямое кодирование (One Hot Encoding)

![img](https://miro.medium.com/max/1200/1*ggtP4a5YaRx6l09KQaYOnw.png)

Можно использовать ColumnTransformer из sklearn.compose и OneHotEncoder из sklearn.preprocessing:  
https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html  
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [13]:
# А можно еще проще: 
X_transform = pd.get_dummies(X)
X_transform

,age,fnlwgt,education.num,sex,capital.gain,capital.loss,hours.per.week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,...,native.country_Portugal,native.country_Puerto-Rico,native.country_Scotland,native.country_South,native.country_Taiwan,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia
1,82,132870,9,0,0,4356,18,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,54,140359,4,0,0,3900,40,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,41,264663,10,0,0,3900,40,0,0,1,...,0,0,0,0,0,0,0,1,0,0
5,34,216864,9,0,0,3770,45,0,0,1,...,0,0,0,0,0,0,0,1,0,0
6,38,150601,6,1,0,3770,40,0,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,310152,10,1,0,0,40,0,0,1,...,0,0,0,0,0,0,0,1,0,0
32557,27,257302,12,0,0,0,38,0,0,1,...,0,0,0,0,0,0,0,1,0,0
32558,40,154374,9,1,0,0,40,0,0,1,...,0,0,0,0,0,0,0,1,0,0
32559,58,151910,9,0,0,0,40,0,0,1,...,0,0,0,0,0,0,0,1,0,0


Большинство алгоритмов машинного обучения ведут себя гораздо лучше, если все признаки измеряются по одной шкале.  
Существуют два общих подхода к приведению разных признаков к одной шкале: нормализация и стандартизация. В различных областях эти термины нередко используются довольно нечетко, и их конкретное содержание приходится выводить из контекста. Чаще всего нормализация означает приведение (нормирование) признаков к диапазону [0, 1] (минимаксное масштабирование, https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html).   
Несмотря на то, что широко используемый метод нормализация путем минимаксного масштабирования целесообразно использовать, когда нам нужны значения в ограниченном интервале, для многих алгоритмов машинного обучения может быть более практичной cтандартизация. Стандартизация набора данных подразумевает такое масштабирование данных, при котором каждый признак имеет среднее значение равное нулю и дисперсию равную единице. Многие линейные модели, такие как логистическая регрессия и метод опорных векторов инициализируют веса нулями либо малыми случайными величинами, близкими к 0. При помощи стандартизации столбцы имеют нормальное распределение, что упрощает извлечение весов. Кроме того, стандартизация содержит полезную информацию о выбросах и делает алгоритм менее к ним чувствительным.

In [14]:
from sklearn.preprocessing import StandardScaler 

# Осуществим стандартизацию
scale = StandardScaler() # Создаем экземпляр класса
X_scale = scale.fit_transform(X_transform) # Преобразуем данные

In [15]:
X_scale

array([[ 3.31662977, -0.53879012, -0.4397382 , ...,  0.31087053,
        -0.04611277, -0.02303802],
       [ 1.18483085, -0.46790594, -2.40055892, ...,  0.31087053,
        -0.04611277, -0.02303802],
       [ 0.19506706,  0.70864459, -0.04757405, ...,  0.31087053,
        -0.04611277, -0.02303802],
       ...,
       [ 0.11893139, -0.33525248, -0.4397382 , ...,  0.31087053,
        -0.04611277, -0.02303802],
       [ 1.48937355, -0.3585745 , -0.4397382 , ...,  0.31087053,
        -0.04611277, -0.02303802],
       [-1.25151078,  0.11070545, -0.4397382 , ...,  0.31087053,
        -0.04611277, -0.02303802]])

In [16]:
y

1        0
3        0
4        0
5        0
6        0
        ..
32556    0
32557    0
32558    1
32559    0
32560    0
Name: income, Length: 30162, dtype: int64

In [17]:
from sklearn.model_selection import train_test_split

# Разбиваем выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size = 0.3, random_state = 42)

In [18]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import classification_report # Позволяет получить сразу несколько метрик

lr = LogisticRegression() # Создаем экземпляр класса
lr.fit(X_train, y_train)  # Обучаем
y_pred = lr.predict(X_test) # Делаем предсказание на тестовой выборке

print(classification_report(y_test, y_pred)) # Оценим качество

              precision    recall  f1-score   support

           0       0.87      0.92      0.90      6754
           1       0.73      0.61      0.66      2295

    accuracy                           0.84      9049
   macro avg       0.80      0.77      0.78      9049
weighted avg       0.84      0.84      0.84      9049



In [19]:
lr.predict_proba(X_test) # Вероятности принадлежности классам

array([[0.44415447, 0.55584553],
       [0.99826432, 0.00173568],
       [0.98676876, 0.01323124],
       ...,
       [0.95960927, 0.04039073],
       [0.9939045 , 0.0060955 ],
       [0.89684099, 0.10315901]])

In [20]:
lr.predict(X_test)

array([1, 0, 0, ..., 0, 0, 0], dtype=int64)